In [12]:
import nltk
nltk.download('punkt')
import math
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
pd.set_option('display.max_colwidth', None)
import re
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, StupidBackoff, Laplace
from nltk.lm import Vocabulary
from nltk.util import ngrams

# df = pd.read_csv("../../../data_raw/kaggle_poem_dataset.csv")
# df.drop(columns=["Unnamed: 0", "Author", "Title", "Poetry Foundation ID"], inplace=True)
# df.rename(columns={"Content": "poem"}, inplace=True)
# df.tail(3)
df = pd.read_csv("../../../data_raw/our_dataset.csv")
df.drop(columns=["Unnamed: 0", "topic"], inplace=True)
df.rename(columns={"Content": "poem"}, inplace=True)
df.tail(3)

[nltk_data] Downloading package punkt to /home/akaagi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,poem
10061,"I imagined the atmosphere would be clear, \nshot with pristine light, \nnot this sulphurous haze, \nthe air ionized as before a thunderstorm. \n\nMany have pictured a river here, \nbut no one mentioned all the boats, \ntheir benches crowded with naked passengers, \neach bent over a writing tablet. \n\nI knew I would not always be a child \nwith a model train and a model tunnel, \nand I knew I would not live forever, \njumping all day through the hoop of myself. \n\nI had heard about the journey to the other side \nand the clink of the final coin \nin the leather purse of the man holding the oar, \nbut how could anyone have guessed \n\nthat as soon as we arrived \nwe would be asked to describe this place \nand to include as much detail as possible— \nnot just the water, he insists, \n\nrather the oily, fathomless, rat-happy water, \nnot simply the shackles, but the rusty, \niron, ankle-shredding shackles— \nand that our next assignment would be \n\nto jot down, off the tops of our heads, \nour thoughts and feelings about being dead, \nnot really an assignment, \nthe man rotating the oar keeps telling us— \n\nthink of it more as an exercise, he groans, \nthink of writing as a process, \na never-ending, infernal process, \nand now the boats have become jammed together, \n\nbow against stern, stern locked to bow, \nand not a thing is moving, only our diligent pens."
10062,"Miss Nancy Ellicott\nStrode across the hills and broke them,\nRode across the hills and broke them—\nThe barren New England hills—\nRiding to hounds\nOver the cow-pasture.\n\nMiss Nancy Ellicott smoked\nAnd danced all the modern dances;\nAnd her aunts were not quite sure how they felt about it,\nBut they knew that it was modern.\n\nUpon the glazen shelves kept watch\nMatthew and Waldo, guardians of the faith,\nThe army of unalterable law."
10063,"In the dark we disappear, pure being.\nOur mirror images, impure being.\n\nBeing and becoming (Heidegger), being and\nnothingness (Sartre)—which is purer being?\n\nBeing alone is no way to be: thus\nloneliness is the test of pure being.\n\nNights in love I fell too far or not quite\nfar enough—one pure, one impure being.\n\nClouds, snow, mist, the dragon's breath on water,\nsmoke from fire—a metaphor's pure being.\n\nStillness and more stillness and the light locked\ndeep inside—both pure and impure being.\n\nIs is the verb of being, I the noun—\nor pronoun for the purists of being.\n\nI was, I am, I looked within and saw\nnothing very clearly: purest being."


In [13]:
tokenized_text = [word_tokenize(poem) for poem in df['poem']]
# Define seed words for text generation
seed_words = ['Sunday','we','lay','hands']

# Iterate over different n-gram sizes
for n in range(1, 7):
    print(f"Generating text for n-gram size: {n}")
    
    # Prepare training data and vocabulary for the current n-gram size
    train_data, vocab = padded_everygram_pipeline(n, tokenized_text)
    
    # Instantiate and train the MLE language model
    mle_lm = MLE(n)
    mle_lm.fit(train_data, vocab)
    
    train_data, vocab = padded_everygram_pipeline(n, tokenized_text)
    # Instantiate and train the Stupid Backoff language model
    sb_lm = StupidBackoff(order=n)
    sb_lm.fit(train_data, vocab)
    
    train_data, vocab = padded_everygram_pipeline(n, tokenized_text)
    # Instantiate and train the Laplace language model
    laplace_lm = Laplace(n)
    laplace_lm.fit(train_data, vocab)
    
    # Generate text using each model
    mle_text = mle_lm.generate(200, text_seed=seed_words, random_seed=42)
    sb_text = sb_lm.generate(200, text_seed=seed_words, random_seed=42)
    laplace_text = laplace_lm.generate(200, text_seed=seed_words, random_seed=42)
    
    # Print the generated text for each model
    print("MLE Text:", " ".join(seed_words), " ".join(mle_text))
    print("\n")
    print("Stupid Backoff Text:", " ".join(seed_words), " ".join(sb_text))
    print("\n")
    print("Laplace Text:", " ".join(seed_words), " ".join(laplace_text))
    print("\n")


Generating text for n-gram size: 1
MLE Text: Sunday we lay hands numerous , along The scatter or to . ever , That his , No of it The make the 'd that—drinking pleased blossoms Euro-päische with bird . . the mind the s invisible you continual its the my they litel problems , This and - Valor . an not cell clan Rome after were of morning I s He corn ’ oar kings over the star.— Till , at again See where time at of depths valley furious across a land about love touch dilation The “ hours . , . next t ever , country “ in years they ) riser out is after of . fewlis from wine till about him I us though and now morning Dismounted snowflake is still in ! be , way to the around , to who . he , small solve ? had it across throng everything She is s Ode as ’ of first in : The blackberries made To The , no Tick-tick two thereon , Which on Some Abyss were light grooves submission the Leave . fate evil gold s only —Anonymous . do bliss they a Laughing for ever an a was foes they itLike , ” the


Stup